In [41]:
!pip install pandas xlsxwriter


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [42]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("records").getOrCreate()

In [43]:
spark.sql("DROP TABLE IF EXISTS demo.nyc.taxis_5_100M")

DataFrame[]

In [44]:
import os
iceberg_table_dir = "../warehouse/nyc/taxis_5_100M"
metadata_dir = f"{iceberg_table_dir}/metadata"
data_dir = f"{iceberg_table_dir}/data"
input_data_dir = f"../input_data"
analysis_info = []
records_before_op = 0

def append_to_file(file_path, msg):
    open_mode = "a"
    if not os.path.exists(file_path):
        open_mode = "w"

    # Open the CSV file in write mode
    with open(file_path, open_mode) as file:
        writer = csv.writer(file)
        
        if open_mode=="w":
            #writing header of the columns
            writer.writerows([list(msg.keys())])    

        row_values = [list(msg.values())]
        # Write the data to the CSV file
        writer.writerows(row_values)

def get_size():
    # List the metadata files
    manifest_pattern = re.compile(r".*-m\d+\.avro$")
    metadata_files = os.listdir(metadata_dir)
    
    # Initialize variables to store the sizes of different types of metadata files
    snap_avro_size = 0
    metadata_json_size = 0
    m_avro_size = 0

    data_dir_size = 0
    # get data dir size
    data_dir_files = os.listdir(data_dir)
    # print(data_dir_files)
    for filename in data_dir_files:
        file_path = os.path.join(data_dir, filename)
        data_dir_size += os.path.getsize(file_path) / 1024  # Convert size to KB
    
    # Iterate through the metadata files and calculate their sizes
    for file in metadata_files:
        file_path = os.path.join(metadata_dir, file)
        file_size_kb = os.path.getsize(file_path) / 1024  # Convert size to KB
        
        if file.startswith("snap-") and file.endswith(".avro"):
            snap_avro_size += file_size_kb
        elif file.endswith(".metadata.json"):
            metadata_json_size += file_size_kb
        elif manifest_pattern.match(file):
            m_avro_size += file_size_kb
    
    # Print the time taken and the sizes of the metadata files
    # print(f"Time taken to read Parquet files: {time_taken:.2f} seconds")
    # print(f"Size of snap-*.avro files: {snap_avro_size:.2f} KB")
    # print(f"Size of *.metadata.json files: {metadata_json_size:.2f} KB")
    # print(f"Size of *m{0-9}{1,}.avro files: {m_avro_size:.2f} KB")

    return {"data_dir_size": data_dir_size,"metadata_size": metadata_json_size,"snapshot_size": snap_avro_size,"manifest_size": m_avro_size}


In [45]:
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType
schema = StructType([
  StructField("vendor_id", LongType(), True),
  StructField("trip_id", LongType(), True),
  StructField("trip_distance", FloatType(), True),
  StructField("fare_amount", DoubleType(), True),
  StructField("store_and_fwd_flag", StringType(), True)
])

df = spark.createDataFrame([], schema)
df.writeTo("demo.nyc.taxis_5_100M").create()

In [46]:
df = spark.table("demo.nyc.taxis_5_100M")
df.show()

+---------+-------+-------------+-----------+------------------+
|vendor_id|trip_id|trip_distance|fare_amount|store_and_fwd_flag|
+---------+-------+-------------+-----------+------------------+
+---------+-------+-------------+-----------+------------------+



## Perform operations

In [47]:
import time, csv
from pyspark.sql.functions import col, when
from pyspark.sql import functions as F
import os

input_data_dir = f"../input_data"
output_dir = f"../output"
analysis_info = []
records_before_op = 0
    
file_type = input("Enter input file type csv or parquet? : ")
file_type = file_type.lower().strip()
input_data_dir = os.path.join(input_data_dir, file_type)
input_files = os.listdir(input_data_dir)

analysis_file = os.path.join(output_dir, f"analysis_info_{file_type}.csv")
if os.path.exists(analysis_file):
    os.remove(analysis_file)

df = spark.table("demo.nyc.taxis_5_100M")
records_before_op = df.count()

digits = len(str(records_before_op))

# Track overall start time
for file in input_files:
    print(f"Started with file={file}")
    file_path = os.path.join(input_data_dir, file)

    # Record the start time for insertion
    insertion_start_time = time.time()
    
    if file_type == "parquet":
        # Read the Parquet files into a DataFrame
        df = spark.read.parquet(file_path)
    else:
        df = spark.read.csv(file_path, header=True)
        df = df.select(
            F.col("vendor_id").cast("long").alias("vendor_id"),
            F.col("trip_id").cast("long").alias("trip_id"),
            F.col("trip_distance").cast("float").alias("trip_distance"),
            F.col("fare_amount").cast("double").alias("fare_amount"),
            F.col("store_and_fwd_flag").cast("string").alias("store_and_fwd_flag")
        )
    
    rows = df.count()
    
    # Write the DataFrame to Parquet format (insert operation)
    df.writeTo("demo.nyc.taxis_5_100M").append()
    
    # Record the end time for insertion
    insertion_end_time = time.time()
    
    # Calculate total insertion time
    insertion_time_taken = insertion_end_time - insertion_start_time

    # Get details
    details = get_size()
    details["time_taken"] = f"{insertion_time_taken:.2f} sec"
    details["Operation"] = f"Inserted {rows} records"
    details["records_after_op"] = records_before_op + rows

    # Add insertion times to the details
    details["insertion_start_time"] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(insertion_start_time))
    details["insertion_end_time"] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(insertion_end_time))
    details["total_insertion_time"] = f"{insertion_time_taken:.2f} sec"

    # Print details before storing to CSV
    print(f"Details before appending to CSV: {details}")
    
    records_before_op += rows
    del insertion_start_time, insertion_end_time, df

    # Store this info in the CSV file
    append_to_file(analysis_file, details)
    analysis_info.append(details)
    
    print(f"Inserted {rows} records..")
    
    ##### Updated records #####
    current_digit = len(str(records_before_op))

    #### Don't perform update operation if the digit has not increased
    if current_digit <= digits:
        continue
    else:
        digits = current_digit
        df = spark.table("demo.nyc.taxis_5_100M")
    
    # Perform the update operation for the first 10 distinct vendor_ids
    for vendor_id in df.select('vendor_id').distinct().collect()[:10]:
        vendor_id = vendor_id[0]
        df = spark.table("demo.nyc.taxis_5_100M")

        st = time.time()
        # Perform an update operation: Set fare_amount to fare_amount+40.0 for vendor_id
        updated_df = df.withColumn("fare_amount", 
                                  when(col("vendor_id") == vendor_id, col("fare_amount")+40)
                                  .otherwise(col("fare_amount")))
        
        # Overwrite the updated DataFrame back to the table
        updated_df.writeTo("demo.nyc.taxis_5_100M").overwritePartitions()
        
        end = time.time() - st
        rows = updated_df.filter(updated_df['vendor_id']==vendor_id).count()

        # Get details
        details = get_size()
        details["time_taken"] = f"{end:.2f} sec"
        details["Operation"] = f"Updated {rows} records"
        details["records_after_op"] = records_before_op

        append_to_file(analysis_file, details)

        del df, st, end
        analysis_info.append(details)

# Function to get the size of the current table (e.g., row count or any other relevant metric)
def get_size():
    # Example: Return the size of the table and other useful metrics.
    # Update this function based on your needs. This is just an example.
    table_size = spark.table("demo.nyc.taxis_5_100M").count()  # Placeholder for table size retrieval
    return {"table_size": table_size}

# Function to append details to the CSV file
def append_to_file(file, details):
    with open(file, mode='a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=details.keys())
        if f.tell() == 0:  # If file is empty, write headers
            writer.writeheader()
        writer.writerow(details)
        print(f"Appended details to {file}: {details}")  # Log the details being appended


Enter input file type csv or parquet? :  parquet


Started with file=records_1000000_part_100_1740479562.9411988.parquet


Details before appending to CSV: {'data_dir_size': 0.5, 'metadata_size': 1.0, 'snapshot_size': 1.0, 'manifest_size': 0.5, 'time_taken': '2.49 sec', 'Operation': 'Inserted 1000000 records', 'records_after_op': 1000000, 'insertion_start_time': '2025-02-25 10:34:46', 'insertion_end_time': '2025-02-25 10:34:49', 'total_insertion_time': '2.49 sec'}
Inserted 1000000 records..


Started with file=records_1000000_part_10_1740479017.2732255.parquet


Details before appending to CSV: {'data_dir_size': 6.0, 'metadata_size': 6.5, 'snapshot_size': 6.5, 'manifest_size': 11.0, 'time_taken': '1.77 sec', 'Operation': 'Inserted 1000000 records', 'records_after_op': 2000000, 'insertion_start_time': '2025-02-25 10:35:11', 'insertion_end_time': '2025-02-25 10:35:12', 'total_insertion_time': '1.77 sec'}
Inserted 1000000 records..
Started with file=records_1000000_part_11_1740479024.0898237.parquet


...

Details before appending to CSV: {'data_dir_size': 103.0, 'metadata_size': 58.5, 'snapshot_size': 58.5, 'manifest_size': 72.5, 'time_taken': '1.79 sec', 'Operation': 'Inserted 1000000 records', 'records_after_op': 96000000, 'insertion_start_time': '2025-02-25 10:40:36', 'insertion_end_time': '2025-02-25 10:40:38', 'total_insertion_time': '1.79 sec'}
Inserted 1000000 records..
Started with file=records_1000000_part_97_1740479544.9426436.parquet


Details before appending to CSV: {'data_dir_size': 103.5, 'metadata_size': 59.0, 'snapshot_size': 59.0, 'manifest_size': 73.0, 'time_taken': '1.82 sec', 'Operation': 'Inserted 1000000 records', 'records_after_op': 97000000, 'insertion_start_time': '2025-02-25 10:40:39', 'insertion_end_time': '2025-02-25 10:40:41', 'total_insertion_time': '1.82 sec'}
Inserted 1000000 records..
Started with file=records_1000000_part_98_1740479550.9369109.parquet


Details before appending to CSV: {'data_dir_size': 104.0, 'metadata_size': 59.5, 'snapshot_size': 59.5, 'manifest_size': 73.5, 'time_taken': '2.05 sec', 'Operation': 'Inserted 1000000 records', 'records_after_op': 98000000, 'insertion_start_time': '2025-02-25 10:40:42', 'insertion_end_time': '2025-02-25 10:40:44', 'total_insertion_time': '2.05 sec'}
Inserted 1000000 records..
Started with file=records_1000000_part_99_1740479556.937529.parquet


Details before appending to CSV: {'data_dir_size': 104.5, 'metadata_size': 60.0, 'snapshot_size': 60.0, 'manifest_size': 74.0, 'time_taken': '1.63 sec', 'Operation': 'Inserted 1000000 records', 'records_after_op': 99000000, 'insertion_start_time': '2025-02-25 10:40:45', 'insertion_end_time': '2025-02-25 10:40:46', 'total_insertion_time': '1.63 sec'}
Inserted 1000000 records..
Started with file=records_1000000_part_9_1740479010.487397.parquet


Details before appending to CSV: {'data_dir_size': 105.0, 'metadata_size': 60.5, 'snapshot_size': 60.5, 'manifest_size': 74.5, 'time_taken': '1.63 sec', 'Operation': 'Inserted 1000000 records', 'records_after_op': 100000000, 'insertion_start_time': '2025-02-25 10:40:47', 'insertion_end_time': '2025-02-25 10:40:49', 'total_insertion_time': '1.63 sec'}
Inserted 1000000 records..
